# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load cities.csv
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()


,City,Latitude,Lng,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
0,Káto Mazaráki,38.07,21.65,20,GR,1603225924,87,60.80,7.00
1,Puerto Ayora,-0.74,-90.35,75,EC,1603225925,69,77.00,11.41
2,Gonbad-e Kāvūs,37.25,55.17,0,IR,1603225926,52,64.81,4.70
3,Coquimbo,-29.95,-71.34,1,CL,1603225867,72,60.80,16.11
4,George Town,5.41,100.34,20,MY,1603225928,88,80.60,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Variables
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Latitude", "Lng"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_vacation_df = cities.loc[(cities["Wind Speed"] <= 10) & 
                               (cities["Cloudiness"] < 10) & 
                               (cities['Humidity']< 35) &
                               (cities["Max Temp"] >= 60) & (cities["Max Temp"] <= 95)].dropna()

ideal_vacation_df.head()

,City,Latitude,Lng,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
38,Bilma,18.69,12.92,0,NE,1603225966,15,81.36,4.79
200,Ocampo,21.65,-101.50,0,MX,1603226157,27,72.43,8.86
274,Tahoua,14.89,5.27,0,NE,1603226254,12,91.09,8.43
284,Najrān,17.49,44.13,0,SA,1603226265,33,71.60,4.70
295,Saint George,37.10,-113.58,1,US,1603226184,12,82.99,2.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create hotel_df
hotel_df = ideal_vacation_df.loc[:,["City","Country", "Latitude", "Lng"]]

# Add column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df


,City,Country,Latitude,Lng,Hotel Name
38,Bilma,NE,18.69,12.92,
200,Ocampo,MX,21.65,-101.50,
274,Tahoua,NE,14.89,5.27,
284,Najrān,SA,17.49,44.13,
295,Saint George,US,37.10,-113.58,
341,Shakawe,BW,-18.37,21.85,
344,Gallup,US,35.53,-108.74,
358,Arlit,NE,18.74,7.39,
366,Taoudenni,ML,22.68,-3.98,
377,Santiago,CL,-33.46,-70.65,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:
for index, row in hotel_df.iterrows():
        
# get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Lng"]
    city_name = row["City"]
    
# add keyword to params dict
    params["location"] = f"{lat},{lng}"

#make API request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Add to Data Frame
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        pass

hotel_df

In [ ]:
# Drop locaitons with no hotel within 5000 meters. 
hotel_df.drop(hotel_df.index[hotel_df['Hotel Name'] == 'NaN'], inplace = True)
hotel_df = hotel_df.reset_index()
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the map
fig.add_layer(markers)

# Display figure
fig